# <span style="color:darkslategray">ABC LLC - Leads for Health Insurance Services </span>

### <span style="color:dimgray">Importing packages</span>

In [3]:
# Importing general packages
import pandas as pd
import dfply

from dfply import *
import numpy as np

# Importing packages for vizualization
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
import seaborn as sns

### <span style="color:dimgray">Loading data</span>

<b>Features of df_bids:</b><br/>
● created: time of the bid<br />
● leadID: unique ID of lead<br />
● max_bid: bid offered by Abc LLC for that particular lead<br />
● won: 1 if their bid was the highest and they won the bid, 0 otherwise

In [ ]:
# Reading Bids csv file
df_bids = pd.read_csv('C:/Users/ZsoltNagy/Desktop/github_projects/ABC_LLC_Leads_Boberdoo/ABC_LLC_Leads/Data/bids_hw.csv')
df_bids.head(5)

<b>Features of df_leads:</b><br />
● leadID: unique ID of the lead<br />
● state_18: the state where the lead lives in<br />
● gender_18: lead's gender, which is 1 if male, and 2 if female<br />
● estimated_household_income_18: this is the lead's estimation of their own household income per year in dollars <br />
● cpa_status_18: 1 for closed leads, 0 for lost leads<br />
● premium_amount_18: value of the contract signed, to be paid monthly in dollars<br />

In [6]:
# Reading Leads csv file
df_leads = pd.read_csv('C:/Users/ZsoltNagy/Desktop/github_projects/ABC_LLC_Leads_Boberdoo/ABC_LLC_Leads/Data/leads_hw.csv')
df_leads.head(5)

   Unnamed: 0              created  leadID  max_bid  won
0           1  2017-12-22 16:08:21       4     22.0    1
1           2  2018-01-18 16:18:19      30      4.0    1
2           3  2018-01-22 16:08:57     120      5.0    1
3           4  2018-01-23 17:28:09     258      5.0    1
4           5  2018-01-23 18:16:10     266      6.0    1
   Unnamed: 0  leadID state_18  gender_18  age_18  \
0           1       4       IL          1   37.00   
1           2      30       NY          2   43.35   
2           3     120       NY          1   32.96   
3           4     258       NY          1   34.82   
4           5     266       NY          2   40.35   

   estimated_household_income_18  cpa_status_18  premium_amount_18  
0                          45000            1.0                700  
1                          65000            0.0                  0  
2                          90000            0.0                  0  
3                          65000            0.0                